<a href="https://colab.research.google.com/github/Ishu2311/prognosAI_project/blob/main/RUL_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RUL Target Computation & Documentation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [5]:
## Load the rolling window sequence numpy array
sequence = np.load('/content/drive/MyDrive/Dataset of ai/rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequence.shape}")

Loaded rolling window sequences with shape: (17731, 30, 66)


In [6]:
## Load the metadata
metadata_df = pd.read_csv('/content/drive/MyDrive/Dataset of ai/sequence_metadata.csv')
print(f"Loaded metadata with shape: {metadata_df.shape}")
metadata_df.shape

Loaded metadata with shape: (17731, 2)


(17731, 2)

In [7]:
## Calculate the RUL per sequence

# Find the max. cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,192.0
1,2,287.0
2,3,179.0
3,4,189.0
4,5,269.0


In [8]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()

,engine_id,cycle,max_cycle
0,1,30.0,192.0
1,1,31.0,192.0
2,1,32.0,192.0
3,1,33.0,192.0
4,1,34.0,192.0


In [9]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [10]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,30.0,192.0,162.0
1,1,31.0,192.0,161.0
2,1,32.0,192.0,160.0
3,1,33.0,192.0,159.0
4,1,34.0,192.0,158.0


In [11]:
# Drop 'max_cycle' column
metadata_df.drop(columns='max_cycle', inplace=True)

In [12]:
metadata_df.head()

,engine_id,cycle,RUL
0,1,30.0,162.0
1,1,31.0,161.0
2,1,32.0,160.0
3,1,33.0,159.0
4,1,34.0,158.0


In [13]:
## Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3] # first 3 engines
for engine in engines_to_check:
  sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
  print(f"\nEngine {engine} - Cycles and RULs:")
  print(sample.head(10))
  print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle    RUL
0   30.0  162.0
1   31.0  161.0
2   32.0  160.0
3   33.0  159.0
4   34.0  158.0
5   35.0  157.0
6   36.0  156.0
7   37.0  155.0
8   38.0  154.0
9   39.0  153.0
     cycle  RUL
153  183.0  9.0
154  184.0  8.0
155  185.0  7.0
156  186.0  6.0
157  187.0  5.0
158  188.0  4.0
159  189.0  3.0
160  190.0  2.0
161  191.0  1.0
162  192.0  0.0

Engine 2 - Cycles and RULs:
     cycle    RUL
163   30.0  257.0
164   31.0  256.0
165   32.0  255.0
166   33.0  254.0
167   34.0  253.0
168   35.0  252.0
169   36.0  251.0
170   37.0  250.0
171   38.0  249.0
172   39.0  248.0
     cycle  RUL
411  278.0  9.0
412  279.0  8.0
413  280.0  7.0
414  281.0  6.0
415  282.0  5.0
416  283.0  4.0
417  284.0  3.0
418  285.0  2.0
419  286.0  1.0
420  287.0  0.0

Engine 3 - Cycles and RULs:
     cycle    RUL
421   30.0  149.0
422   31.0  148.0
423   32.0  147.0
424   33.0  146.0
425   34.0  145.0
426   35.0  144.0
427   36.0  143.0
428   37.0  142.0
429   38.0  141.0
430   3

In [14]:
## Save updated metadata with RUL target
metadata_df.to_csv('sequence_metadata_with_RUL.csv', index=False)

**Process Steps:**  
- Loaded rolling window feature sequences from 'rolling_windows_feature.npy'.  
- Loaded sequence metadata from 'sequence_metadata.csv' including engine and cycle identifiers.  
- Computed Remaining Useful Life (RUL) for each sequence as the difference between
  maximum cycle for the engine and the current cycle.    
- Saved updated metadata including RUL targets for downstream model training.

**Outputs:**  
- Rolling window sequences (NumPy array) ready for modeling.  
- Updated sequence metadata with assigned RUL targets.  

**Next Steps:**  
- Use these datasets for training and evaluating LSTM/GRU predictive models for RUL.  

---